In [12]:
import os
import shutil
import random

In [11]:
# Path ke folder yang berisi gambar-gambar
source_folder = 'images'

# List semua file dalam folder
files = os.listdir(source_folder)

# Loop melalui setiap file
for file in files:
    # Split nama file untuk mendapatkan nama dan ekstensi
    filename, file_extension = os.path.splitext(file)
    
    # Split nama untuk mendapatkan kategori dan nomor
    category, number = filename.split('_')
    
    # Reformat kategori dengan mengganti '+' menjadi spasi
    category = category.replace('+', ' ')
    
    # Path ke folder tujuan
    destination_folder = os.path.join(source_folder, category)
    
    # Buat folder tujuan jika belum ada
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Path lengkap ke file sumber
    source_path = os.path.join(source_folder, file)
    
    # Path lengkap ke file tujuan
    destination_path = os.path.join(destination_folder, file)
    
    # Pindahkan file ke folder tujuan
    shutil.move(source_path, destination_path)
    
print("Organisasi selesai.")

Organisasi selesai.


In [7]:
import os
import requests
import base64
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Read the Excel file containing food names
df = pd.read_excel("data2/Food 101 prep.xlsx")

def get_google_images(query, max_images=50):
    query = query.replace(' ', '+')
    url = f"https://www.google.com/search?udm=2&q={query}"

    driver.get(url)
    driver.maximize_window()
    driver.execute_script("window.scrollBy(0, 4000)")
    time.sleep(10)
    images = driver.find_elements(By.XPATH, '//img[@class="YQ4gaf"]')
    idx = 0
    total_images = 0  # Inisialisasi total gambar yang sudah diambil

    for _, image in enumerate(images):
        if total_images >= max_images:  # Berhenti jika jumlah gambar sudah mencapai max_images
            break

        image_src = image.get_attribute('src')
        if image_src.startswith('data:image/jpeg'):
            try:
                if os.path.exists(f"images/{query}_{idx+1}.jpg") and os.path.getsize(f"images/{query}_{idx+1}.jpg") > 0:
                    print(f'File {query}_{idx+1}.jpg sudah ada')
                    idx += 1
                    continue
            except FileNotFoundError:
                pass
            
            header, base64_data = image_src.split(',', 1)
            image_data = base64.b64decode(base64_data)
            with open(os.path.join('images', f"{query}_{idx+1}.jpg"), 'wb') as handler:
                handler.write(image_data)
            idx += 1
            total_images += 1  # Menambah total gambar yang sudah diambil
            print(f'jpeg Successfully scraped and saved {query}_{idx+1}.jpg')
        elif image_src.startswith('https://'):
            try:
                if os.path.exists(f"images/{query}_{idx+1}.jpg") and os.path.getsize(f"images/{query}_{idx+1}.jpg") > 0:
                    print(f'File {query}_{idx+1}.jpg sudah ada')
                    idx += 1
                    continue
            except FileNotFoundError:
                pass
                
            img_data = requests.get(image_src).content
            with open(os.path.join('images', f"{query}_{idx+1}.jpg"), 'wb') as handler:
                handler.write(img_data)
            idx += 1
            total_images += 1  # Menambah total gambar yang sudah diambil
            print(f'https Successfully scraped and saved {query}_{idx+1}.jpg')



def scrape_images():
    # Set up Chrome webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Iterate through food names and scrape images
    for food_name in df["Tambahan (Web Scraping)"]:
        idx = 0  # Initialize index for each food name
        # Check if food_name is not NaN and convert it to string before stripping
        if pd.notna(food_name):
            get_google_images(str(food_name).strip(), max_images=50)

    driver.quit()

# Create 'images' directory if it doesn't exist
os.makedirs('images', exist_ok=True)

# Call the function to scrape images
scrape_images()


File Ikan+Kembung+Goreng_1.jpg sudah ada
File Ikan+Kembung+Goreng_2.jpg sudah ada
File Ikan+Kembung+Goreng_3.jpg sudah ada
File Ikan+Kembung+Goreng_4.jpg sudah ada
File Ikan+Kembung+Goreng_5.jpg sudah ada
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_7.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_8.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_9.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_10.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_11.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_12.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_13.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_14.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_15.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_16.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_17.jpg
jpeg Successfully scraped and saved Ikan+Kembung+Goreng_18.jpg
jpeg Successfully scraped and saved Ikan+K

In [ ]:
def split_dataset(data_dir, train_dir, validation_dir, split_ratio=0.8):
    # Membuat folder training dan validation jika belum ada
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    
    # Loop untuk setiap kelas (folder makanan)
    for category in os.listdir(data_dir):
        category_path = os.path.join(data_dir, category)
        
        if os.path.isdir(category_path):
            images = os.listdir(category_path)
            random.shuffle(images)  # Mengacak urutan gambar
            
            # Hitung jumlah gambar untuk training dan validation
            train_count = int(len(images) * split_ratio)
            train_images = images[:train_count]
            validation_images = images[train_count:]
            
            # Membuat folder kelas di dalam train dan validation directory
            train_category_dir = os.path.join(train_dir, category)
            validation_category_dir = os.path.join(validation_dir, category)
            os.makedirs(train_category_dir, exist_ok=True)
            os.makedirs(validation_category_dir, exist_ok=True)
            
            # Memindahkan gambar ke folder train dan validation
            for image in train_images:
                shutil.move(os.path.join(category_path, image), os.path.join(train_category_dir, image))
            
            for image in validation_images:
                shutil.move(os.path.join(category_path, image), os.path.join(validation_category_dir, image))
            
            print(f"Kategori: {category} - Training: {len(train_images)} gambar, Validation: {len(validation_images)} gambar")

# Path ke folder dataset asli
data_dir = 'data'

# Path ke folder untuk dataset training dan validation
train_dir = 'data/train'
validation_dir = 'data/validation'

# Memanggil fungsi untuk membagi dataset
split_dataset(data_dir, train_dir, validation_dir)

## scrap bing data

In [14]:
from bing_image_downloader import downloader
downloader.download('Udang Goreng', limit=300, output_dir="images/Udang Goreng")

[%] Downloading Images to d:\Fauzan\projects\machine-learning-bangkit-capstone\Image-Recognition\images\Udang Goreng\Udang Goreng


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://portalmadura.com/wp-content/uploads/2019/07/Menu-Praktis-Udang-Goreng-Bumbu-Bawang-Ketumbar-Super-Gurih.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://3.bp.blogspot.com/-63jtL2GUTk0/UqoxXD78M1I/AAAAAAAABD8/9v6z0BjLR48/s1600/udang+goreng+gurih.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://www.cilibangi.com/app/uploads/sites/2/2015/06/udang-goreng.jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://3.bp.blogspot.com/-bO-SWww8NMk/UWTp5blvRPI/AAAAAAAACw4/tSzsw0DDZGQ/s1600/Udang+Goreng+Renyah+2.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://s0.bukalapak.com/img/090865397/w-1000/Udang_Goreng_Tepung.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://2.bp.blogspot.com/-PbMP5qsjpt8/VUilHlFhmG

: 